# Modélisation thématique
Dans ce notebook, nous effectuons de la modélisation thématique de textes à l'aide de modules Python spécialisés.

**IMPORTANT**: ce notebook requiert le module `java`. S'il n'était pas chargé au moment d'ouvrir ce notebook, vous devez le fermer, l'arrêter, charger le module `java` et rouvrir le présent notebook.

In [ ]:
!which java

## Chargement des modules Python requis

In [ ]:
# Modules réguliers et scientifiques
print('- Chargement des modules réguliers...')
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
from pathlib import Path
import json

# NLTK - Natural Language Toolkit
print('- Chargement de NLTK...')
import nltk
nltk.download('stopwords')  # Requis seulement une fois

# Gensim
print('- Chargement de Gensim...')
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LdaMulticore

# spaCy pour la lemmatisation
print('- Chargement de spaCy...')
import spacy

# Outils de visualisation
print('- Chargement des outils de visualisation...')
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

# Configurer la journalisation de Gensim (optionnel)
print('- Configuration finale...')
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print('Chargement des modules terminé.')

## Chargement des données
* Charger les "mots vides" de la langue française à partir du module NLTK

In [ ]:
# Mots vides dans NLTK sont les "stopwords"
from nltk.corpus import stopwords

stop_words = stopwords.words('french')

# Afficher la liste par défaut
print('Liste par défaut:\n', stop_words)

# Ajouter d'autres mots à la liste, au besoin
stop_words.extend([])

# Afficher la liste finale
print('\nListe finale:\n', stop_words)

* Obtenir la liste des fichiers texte

In [ ]:
# Obtenir le chemin vers tous les fichiers texte dans le dossier "donnees/"
txt_folder = Path('donnees/').rglob('*.txt')

files = sorted([x for x in txt_folder])  # Convertir le tout en une liste triée
print(files[:3], '...', files[-3:])  # Afficher les premiers et derniers fichiers
print(f' => {len(files)} fichiers au total')

* Créer un dictionnaire qui servira à initialiser un DataFrame Pandas avec deux colonnes :
  * `target_names`: le nom du fichier et son chemin
  * `content`: le texte original du fichier regroupé en une seule ligne

In [ ]:
text_dict = {'target_names': [], 'content': []}

# Pour chaque fichier texte
for name in files:
    f = open(name, 'r', encoding='utf-8')
    basename = os.path.basename(name)

    # Afficher la progression à tous les 10 fichiers
    if name in files[::10]:
        print(f'Reading {basename} ...')

    # Noter le nom du fichier et son contenu
    text_dict['target_names'].append(basename)
    text_dict['content'].append(' '.join(f.readlines()))
    f.close()

# Convertir le dictionnaire en dataframe pandas
df = pd.DataFrame.from_dict(text_dict)
print(f'Total: {len(df)} rangées. Voici les 5 premières:')
df.head()

## Nettoyer les données textuelles
* Enlever les chiffres romains et les espaces multiples

In [ ]:
# Sélectionner le contenu de tous les fichiers
data = text_dict['content']

# Supprimer les chiffres romains
data = [re.sub('[MDCLXVI]+(\.|\b\w\n)', ' ', sentence) for sentence in data]

# Remplacer les espaces (et sauts de ligne) multiples par un simple espace
data = [re.sub('\s+', ' ', sentence) for sentence in data]

# Supprimer les caractères de citations
#data = [re.sub("\'", "", sentence) for sentence in data]

print(f'Premier texte nettoyé:\n {data[0][:308]}...\n')
print(f'Dernier texte nettoyé:\n {data[-1][:308]}...')

* Enlever tous les symboles de ponctuation et transformer chaque texte en liste de mots

In [ ]:
def sentences_to_words(sentences):
    """
    Générateur - Pour chaque texte, retourner une liste de mots

    Retourne:
    ---------
    Chaque texte est traité par gensim.utils.simple_preprocess() qui
    enlève la ponctuation et collecte tous les mots individuels.
    """
    for sentence in sentences:
        # L'option deacc=True enlève les symboles de ponctuation
        yield(simple_preprocess(sentence, deacc=True))

# Créer une liste de listes de mots - une liste de mots par texte
data_words = list(sentences_to_words(data))

print('Première liste de mots:\n', data_words[0][:50], '...\n')
print('Dernière liste de mots:\n', data_words[-1][:50], '...')

## Modélisation thématique
On commence par utiliser:
* [la classe Phrases](https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases) de Gensim - détecte les phrases en fonction des décomptes de collocation
* [la classe Phraser](https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phraser) (alias de [FrozenPhrases](https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.FrozenPhrases)) de Gensim - réduit la consommation de mémoire-vive en éliminant les informations optionnelles pour la détection de phrases

In [ ]:
# Construire les modèles bigramme et trigramme - threshold élevé => moins de phrases
bigram = gensim.models.phrases.Phrases(data_words, min_count=4, threshold=8)
trigram = gensim.models.phrases.Phrases(bigram[data_words], threshold=8)

# Moyen plus rapide d'obtenir une phrase identifiée comme un trigramme/bigramme
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Voir l'exemple d'un trigramme
for mot in trigram_mod[bigram_mod[data_words[0]]]:
    if len(mot.split('_')) == 3:
        print(mot)

* Définir des fonctions pour traiter les mots vides, les bigrammes, les trigrammes et la lemmatisation

In [ ]:
def remove_stopwords(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

* Compléter le nettoyage des listes de mots

In [ ]:
print('- Supprimer les mots vides...')
data_words_nostops = remove_stopwords(data_words)

print('- Former les bigrammes...')
data_words_bigrams = make_bigrams(data_words_nostops)

print('- Former les trigrammes...')
data_words_trigrams = make_trigrams(data_words_bigrams)

# Initialiser le modèle spaCy 'fr', en ne gardant que le composant "tagger"
print('- Initialiser le modèle spaCy...')
nlp = spacy.load('fr_core_news_sm', disable=['parser', 'ner'])

# Faire la lemmatisation en ne gardant que les noms, adjectifs, verbes et adverbes
print('- Lemmatisation...')
data_lemmatized = lemmatization(data_words_trigrams,
                                allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0][:50])

* Création du dictionnaire et du corpus

In [ ]:
# Créer le dictionnaire
id2word = corpora.Dictionary(data_lemmatized)

# Calculer la fréquence des mots par fichier
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

# Format lisible d'un extrait du corpus
[[(id2word[id], freq) for id, freq in cp[:10]] for cp in corpus[:4]]

In [ ]:
start = 2   # Le nombre minimum de thèmes par modèle
limit = 10  # Le nombre maximum de thèmes par modèle
step = 2    # Le pas d'augmentation du nombre de thèmes
multiple_num_topics = range(start, limit + 1, step)

model_list = []
coherence_values = []

for num_topics in multiple_num_topics:
    print(f'Avec {num_topics} thèmes...')

    model = LdaMulticore(
        corpus=corpus,
        num_topics=num_topics,
        id2word=id2word,
        workers=1)
    model_list.append(model)

    coherencemodel = CoherenceModel(
        model=model,
        texts=data_lemmatized,
        dictionary=id2word,
        coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())

print('Terminé')

In [ ]:
# Afficher le graphique des valeurs de cohérence
plt.plot(multiple_num_topics, coherence_values)

plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')

plt.show()

In [ ]:
# Afficher les valeurs de cohérence
for m, cv in zip(multiple_num_topics, coherence_values):
    print(f'Pour un nombre de thèmes = {m:2d},',
          f'on obtient une cohérence de {round(cv, 4)}')

In [ ]:
# Choissisez le modèle que vous croyez être le meilleur
# Rappel - les indices commencent à 0 dans Python
optimal_model = model_list[3]

# Affichage des différents thèmes
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
# Relancer le modèle avec le nombre exact de thèmes
ldamallet = LdaMulticore(corpus=corpus, num_topics=8, id2word=id2word, workers=1)

In [ ]:
# Afficher les thèmes retenus
pprint(ldamallet.show_topics(formatted=False))

# Afficher la cohérance
coherence_model_ldamallet = CoherenceModel(
    model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nScore de cohérence: ', coherence_ldamallet)

In [ ]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df):
    # Créer un nouveau DataFrame
    sent_topics_df = pd.DataFrame()

    # Extraire les thèmes principaux de chaque document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)

        # Obtenir le Dominant_Topic, le Perc_Contribution et les Topic_Keywords
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => thème principal
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(
                    pd.Series(
                        [int(topic_num), round(prop_topic,4), topic_keywords]),
                    ignore_index=True)
            else:
                break

    sent_topics_df.columns = [
        'Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Ajouter les colonnes nom de fichier et contenu
    contents = texts
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
# Préparer les résultats finaux
df_topic_sents_keywords = format_topics_sentences(
    ldamodel=ldamallet, corpus=corpus, texts=df)

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = [
    'Document number',
    'Dominant_Topic',
    'Topic_Perc_Contrib',
    'Keywords',
    'file_name',
    'Text']

In [ ]:
# Afficher les résultats finaux
df_dominant_topic